In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split

# Load data
data = np.loadtxt('/content/drive/MyDrive/BarbellAI/barbell_data.csv', delimiter=',', skiprows=1)
X = data[:, :6]  # ax, ay, az, gx, gy, gz
y = data[:, 6:8] # rep_label, exercise_id

# Normalize
X = (X - np.mean(X, axis=0)) / np.std(X, axis=0)

# Reshape for LSTM (samples, timesteps, features)
X = X.reshape(-1, 50, 6) # 50-step sequences

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Build hybrid model
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(50, 6)),
    tf.keras.layers.Conv1D(32, 5, activation='relu'),
    tf.keras.layers.LSTM(64, return_sequences=True),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')  # Output: [rep_prob, vel_correction, exercise_id]
])

# Compile
model.compile(optimizer='adam',
              loss=['binary_crossentropy', 'mse', 'sparse_categorical_crossentropy'],
              metrics=['accuracy'])

# Train (enable GPU: Runtime > Change runtime type > GPU)
model.fit(X_train, [y_train[:,0], y_train[:,1], y_train[:,2]],
          epochs=50,
          batch_size=32,
          validation_data=(X_test, [y_test[:,0], y_test[:,1], y_test[:,2]]))

# Convert to TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

# Save and download
with open('rep_detection_model.tflite', 'wb') as f:
    f.write(tflite_model)

# Convert to C header for Arduino
!apt-get install xxd
!xxd -i rep_detection_model.tflite > rep_detection_model.h

<ipython-input-6-0f471982c336>:6: UserWarning: loadtxt: input contained no data: "/content/drive/MyDrive/BarbellAI/barbell_data.csv"
  data = np.loadtxt('/content/drive/MyDrive/BarbellAI/barbell_data.csv', delimiter=',', skiprows=1)


IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

In [ ]:
from google.colab import files
files.download('rep_detection_model.h')